In [1]:
# Importing libraries

import numpy as np
import pandas as pd
from pandas import ExcelWriter

In [2]:
# Reading companies data set

companies = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1") # Using tab separator as the provided companies.txt file has values separated using tab

companies.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [3]:
# Getting general info of the data set and also to see if there are any missing values

companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [4]:
# % of null values in each column of the companies data set

round(100*(companies.isnull().sum()/len(companies.index)), 2)

permalink         0.00
name              0.00
homepage_url      7.62
category_list     4.74
status            0.00
country_code     10.48
state_code       12.88
region           12.10
city             12.10
founded_at       22.93
dtype: float64

There seems to be a high percentage of nulls in the columns homepage_url, state_code, region, city & founded_at which we can safely ignore as we won't be using them for our analysis.

Nulls in category_list & country_code are going to be handled after rounds2 data set is merged with companies data set.

In [5]:
# Reading rounds2 data set

rounds2 = pd.read_csv("rounds2.csv", sep=",", encoding = "ISO-8859-1") # Using , separator as the provided rounds2.csv file has values separated using comma

rounds2.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [6]:
# Getting general info of the data set and also to see if there are any missing values

rounds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [7]:
# % of null values in each column of the rounds2 data set

round(100*(rounds2.isnull().sum()/len(rounds2.index)), 2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

There seems to be a high percentage of null values in funding_round_code column which we can safely ignore for now as it is not going to be used in the analysis.

We will be handling the nulls in raised_amount_usd column post merge of companies and rounds2 data sets.

# Checkpoint 1: Data Cleaning 1

Before merging the data sets, we need to make the format of permalink columns in each data set uniform so that the inner join doesn't miss out values due to case sensitivity

In [8]:
# Converting the permalink column data in both data sets to lower case to fix the duplicates

rounds2['company_permalink'] = rounds2['company_permalink'].apply(lambda x: x.lower())

companies['permalink'] = companies['permalink'].apply(lambda x: x.lower())

In [9]:
# Decoding the permalink columns data in both data sets using appropriate formats

rounds2.company_permalink = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')

companies.permalink = companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')

In [10]:
# How many unique companies are present in rounds2?

len(pd.unique(rounds2['company_permalink']))

# Can also be found using => len(rounds2['company_permalink'].value_counts(dropna=False)

66368

In [11]:
# How many unique companies are present in companies?

len(pd.unique(companies['permalink']))

# Can also be found using => len(companies['permalink'].value_counts(dropna=False))

66368

In [12]:
# Are there any companies in the rounds2 file which are not present in companies?

rounds2_set = set(rounds2['company_permalink'])
companies_set = set(companies['permalink'])

rounds2_set - companies_set # Using set operations to find the difference

# Can also be found using => rounds2[rounds2['company_permalink'].apply(lambda x: x not in companies['permalink'].values)]

set()

In [13]:
# Also providing the companies that are in companies but not in rounds2

companies_set - rounds2_set # Using set operations to find the difference

set()

In [14]:
# Merging companies data set into rounds2 data set using inner join/merge

master_frame = pd.merge(rounds2, companies, how = 'inner', left_on = 'company_permalink', right_on = 'permalink')

master_frame.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007


In [15]:
# How many observations are present in master_frame?

master_frame.info() # Also to find the general info and null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114949 entries, 0 to 114948
Data columns (total 16 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
permalink                  114949 non-null object
name                       114948 non-null object
homepage_url               108815 non-null object
category_list              111539 non-null object
status                     114949 non-null object
country_code               106271 non-null object
state_code                 104003 non-null object
region                     104782 non-null object
city                       104785 non-null object
founded_at                 94428 non-null object
dtypes: float64(1), object(15)
memory usage: 14.9+ MB


We can proceed with dropping the columns that are not needed for the analysis from the master_frame data set

In [16]:
# Drop funding_round_permalink, permalink, funding_round_code, funded_at, homepage_url, state_code, region, city, founded_at columns from master_frame

drop_col = ['funding_round_permalink','permalink','funding_round_code','funded_at','homepage_url','state_code','region','city','founded_at']

master_frame.drop(drop_col, axis=1, inplace=True)

master_frame.isnull().sum(axis=0) # Find the sum of missing values in each column in master_frame data set

company_permalink         0
funding_round_type        0
raised_amount_usd     19990
name                      1
category_list          3410
status                    0
country_code           8678
dtype: int64

In [17]:
# % of null values in each column of the master_frame data set

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

company_permalink      0.00
funding_round_type     0.00
raised_amount_usd     17.39
name                   0.00
category_list          2.97
status                 0.00
country_code           7.55
dtype: float64

Since raised_amount_usd, name, category_list & country_code are each critical for the analysis, we can't have them as nulls.

Now, null values of name, category_list & country_code can't be imputed using statistical measures as they are not numeric attributes but also can't be assumed to have some value among the existing ones. The logical way to handle them is to delete the rows with null values.

Now, null values of raised_amount_usd column can't be imputed using statistical measures as it would be inaccurate since it depends on multiple attributes like category, country etc. The logical way to handle them is to delete the rows with null values.

In [18]:
# Drop null values of raised_amount_usd, name, category_list, country_code

master_frame = master_frame[pd.notnull(master_frame['raised_amount_usd'])]
master_frame = master_frame[pd.notnull(master_frame['name'])]
master_frame = master_frame[pd.notnull(master_frame['category_list'])]
master_frame = master_frame[pd.notnull(master_frame['country_code'])]

In [19]:
# % of null values in each column of the master_frame data set

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

company_permalink     0.0
funding_round_type    0.0
raised_amount_usd     0.0
name                  0.0
category_list         0.0
status                0.0
country_code          0.0
dtype: float64

In [20]:
master_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88528 entries, 0 to 114947
Data columns (total 7 columns):
company_permalink     88528 non-null object
funding_round_type    88528 non-null object
raised_amount_usd     88528 non-null float64
name                  88528 non-null object
category_list         88528 non-null object
status                88528 non-null object
country_code          88528 non-null object
dtypes: float64(1), object(6)
memory usage: 5.4+ MB


In [21]:
'''
writer = ExcelWriter('/Users/sdittaka/Downloads/master_frame.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
master_frame.to_excel(writer,'Sheet1')
writer.save()
'''

# Checkpoint 2: Funding Type Analysis

In [22]:
# Number of values for each funding round type

master_frame.funding_round_type.value_counts()

venture                  47809
seed                     21094
debt_financing            6506
angel                     4400
grant                     1939
private_equity            1820
undisclosed               1345
convertible_note          1320
equity_crowdfunding       1128
post_ipo_equity            598
product_crowdfunding       330
post_ipo_debt              151
non_equity_assistance       60
secondary_market            28
Name: funding_round_type, dtype: int64

In [23]:
# Obtaining average funding amount for the interested funding types

print("Venture type average funding amount is {} Million USD ".format(str(master_frame.loc[master_frame.funding_round_type == 'venture'].raised_amount_usd.mean()/1000000)))
print("Angel type average funding amount is {} Million USD ".format(str(master_frame.loc[master_frame.funding_round_type == 'angel'].raised_amount_usd.mean()/1000000)))
print("Seed type average funding amount is {} Million USD ".format(str(master_frame.loc[master_frame.funding_round_type == 'seed'].raised_amount_usd.mean()/1000000)))
print("Private Equity type average funding amount is {} Million USD ".format(str(master_frame.loc[master_frame.funding_round_type == 'private_equity'].raised_amount_usd.mean()/1000000)))

Venture type average funding amount is 11.724222690204773 Million USD 
Angel type average funding amount is 0.9715738911363636 Million USD 
Seed type average funding amount is 0.7478279478524699 Million USD 
Private Equity type average funding amount is 73.93848627527473 Million USD 


Since, Spark Funds wants to invest between 5 to 15 million USD per investment round, "venture" is the funding type that is most suitable for it's investment as it's average 11.72 Million USD falls in this range

# Checkpoint 3: Country Analysis

In [24]:
# Obtaining all the observations for the funding type 'venture'

venture_frame = master_frame.loc[master_frame.funding_round_type == 'venture']

venture_frame.head()

,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code
0,/organization/-fame,venture,10000000.0,#fame,Media,operating,IND
4,/organization/0-6-com,venture,2000000.0,0-6.com,Curated Web,operating,CHN
8,/organization/0ndine-biomedical-inc,venture,719491.0,Ondine Biomedical Inc.,Biotechnology,operating,CAN
10,/organization/0xdata,venture,20000000.0,H2O.ai,Analytics,operating,USA
11,/organization/0xdata,venture,1700000.0,H2O.ai,Analytics,operating,USA


In [28]:
# Using pivot table on the venture data frame using sum aggregate function to obtain the top 9 countries based on the total investment amount each country received

top9 = venture_frame.pivot_table(values = 'raised_amount_usd', index = 'country_code', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)[0:9]

top9.raised_amount_usd = top9.raised_amount_usd/1000000

top9.rename(columns={"raised_amount_usd":"raised_amount_usd (in millions)"}, inplace=True)

top9

,raised_amount_usd (in millions)
country_code,
USA,420068.029342
CHN,39338.918773
GBR,20072.813004
IND,14261.508718
CAN,9482.217668
FRA,7226.851352
ISR,6854.350477
DEU,6306.921981
JPN,3167.647127


From http://www.emmir.org/fileadmin/user_upload/admission/Countries_where_English_is_an_official_language.pdf we can confirm that USA, GBR & IND are the Top 3 English-Speaking Countries

In [26]:
writer = ExcelWriter('/Users/sdittaka/Downloads/top9.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
top9.to_excel(writer,'Sheet1')
writer.save()

# Checkpoint 4: Sector Analysis 1

In [29]:
# Reading mapping data set

mapping = pd.read_csv("mapping.csv", sep=",", encoding = "ISO-8859-1") # Using , separator as the provided companies.txt file has values separated using comma

mapping.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [30]:
# Getting general info of the data set and also to see if there are any missing values

mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
category_list                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


There is one missing value (which happens to be the first row in the data set) in the category_list column and that corresponds to the 'Blanks' column. Since, this row and 'Blanks' column are not useful for the analysis, proceeding with dropping them from the data set.

In [31]:
mapping = mapping.drop(columns=['Blanks']) # Drop 'Blanks' column
mapping = mapping.loc[1:687,:] # Skipping the first row

mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 1 to 687
Data columns (total 9 columns):
category_list                              687 non-null object
Automotive & Sports                        687 non-null int64
Cleantech / Semiconductors                 687 non-null int64
Entertainment                              687 non-null int64
Health                                     687 non-null int64
Manufacturing                              687 non-null int64
News, Search and Messaging                 687 non-null int64
Others                                     687 non-null int64
Social, Finance, Analytics, Advertising    687 non-null int64
dtypes: int64(8), object(1)
memory usage: 48.4+ KB


In [32]:
# Examining the data frame

mapping.head()

,category_list,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
1,3D,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,1,0,0,0,0


In [33]:
# Obtaining the main_sector for each row where there is a value 1 into a new data frame with the corresponding category

mapping_new = pd.DataFrame(index=range(len(mapping)), columns=['category_list','main_sector'])

for index, row in mapping.iterrows():
    mapping_new.loc[index] = [row['category_list'], str(row[row == 1].index[0])]

mapping_new.head()

,category_list,main_sector
0,NaN,NaN
1,3D,Manufacturing
2,3D Printing,Manufacturing
3,3D Technology,Manufacturing
4,Accounting,"Social, Finance, Analytics, Advertising"


In [34]:
mapping_new = mapping_new.loc[1:687,:] # To get rid of the first NaN row

mapping_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 1 to 687
Data columns (total 2 columns):
category_list    687 non-null object
main_sector      687 non-null object
dtypes: object(2)
memory usage: 16.1+ KB


By carefully examining the raw mapping data set, we can see that the strings "na" and "Na" has been replaced with value 0 which resulted in values like "Analytics" shown as "A0lytics".

To handle this, we need to replace the occurences of 0 only in the string portion of the category_list values with "na" string.

To make it uniform, we convert the category_list to lower case values.

In [35]:
mapping_new['category_list'] = mapping_new['category_list'].apply(lambda x : x.lower())

mapping_new['category_list'] = mapping_new['category_list'].apply(lambda x : x.replace("0","na") if not ".0" in x else x)

mapping_new.head()

,category_list,main_sector
1,3d,Manufacturing
2,3d printing,Manufacturing
3,3d technology,Manufacturing
4,accounting,"Social, Finance, Analytics, Advertising"
5,active lifestyle,Health


In [36]:
master_frame.category_list = master_frame.category_list.astype('str') # Converting the data type to string to perform string operations

master_frame['primary_sector'] = master_frame.category_list.apply(lambda x: x.split('|')[0]) # Extracting the primary sector of each category list from the category_list column

master_frame['primary_sector'] = master_frame['primary_sector'].apply(lambda x: x.lower()) # Converting to lower case for uniformity

master_frame.head()

,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code,primary_sector
0,/organization/-fame,venture,10000000.0,#fame,Media,operating,IND,media
2,/organization/-qounter,seed,700000.0,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,application platforms
4,/organization/0-6-com,venture,2000000.0,0-6.com,Curated Web,operating,CHN,curated web
6,/organization/01games-technology,undisclosed,41250.0,01Games Technology,Games,operating,HKG,games
7,/organization/0ndine-biomedical-inc,seed,43360.0,Ondine Biomedical Inc.,Biotechnology,operating,CAN,biotechnology


In [37]:
# Finding the categories that are in master frame but not in mapping as these observations are not going to be part of our sector analysis

master_frame_set = set(master_frame['primary_sector'])
mapping_set = set(mapping_new['category_list'])

master_frame_set - mapping_set

{'adaptive equipment',
 'biotechnology and semiconductor',
 'cause marketing',
 'deep information technology',
 'english-speaking',
 'enterprise hardware',
 'generation y-z',
 'golf equipment',
 'google glass',
 'greentech',
 'group email',
 'internet technology',
 'internet tv',
 'kinect',
 'lingerie',
 'mobile emergency&health',
 'natural gas uses',
 'nightlife',
 'product search',
 'psychology',
 'racing',
 'rapidly expanding',
 'real estate investors',
 'registrars',
 'retirement',
 'sex industry',
 'skill gaming',
 'social media advertising',
 'spas',
 'specialty retail',
 'sponsorship',
 'subscription businesses',
 'swimming',
 'toys',
 'vacation rentals'}

In [38]:
# Finding the categories that are in mapping frame but not in master_frame as these observations are not going to be part of our sector analysis

mapping_set - master_frame_set

{'energy storage',
 'film distribution',
 'flowers',
 'lotteries',
 'low bid auctions',
 'multi-level marketing',
 'pervasive computing',
 'plumbers',
 'smart grid',
 'social recruiting',
 'taxis',
 'veterinary',
 'virtual workforces',
 'women'}

In [39]:
# Examining the number of observations that are going to be missed as part of the merge

merged_frame = pd.merge(master_frame, mapping_new, how = 'left', left_on = 'primary_sector', right_on = 'category_list')

merged_frame.isnull().sum(axis=0)

company_permalink      0
funding_round_type     0
raised_amount_usd      0
name                   0
category_list_x        0
status                 0
country_code           0
primary_sector         0
category_list_y       63
main_sector           63
dtype: int64

In [40]:
# Examining the observations that are going to be missed as part of the merge

merged_frame.loc[merged_frame.main_sector.isnull()].loc[:,['primary_sector','main_sector']].sort_values(by='primary_sector').drop_duplicates()

,primary_sector,main_sector
64733,adaptive equipment,NaN
33669,biotechnology and semiconductor,NaN
1995,cause marketing,NaN
85941,deep information technology,NaN
6358,english-speaking,NaN
13745,enterprise hardware,NaN
10772,generation y-z,NaN
85061,golf equipment,NaN
34904,google glass,NaN
67761,greentech,NaN


In [41]:
# Merge master_frame with mapping_new on sector using inner join

master_frame = pd.merge(master_frame, mapping_new, how = 'inner', left_on = 'primary_sector', right_on = 'category_list')
master_frame.drop('category_list_y', axis=1, inplace=True) # Dropping the duplicate category list column
master_frame.rename(columns={"category_list_x":"category_list"}, inplace=True) # Rename to appropriate name
master_frame.head()

,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector
0,/organization/-fame,venture,10000000.0,#fame,Media,operating,IND,media,Entertainment
1,/organization/90min,venture,15000000.0,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,media,Entertainment
2,/organization/90min,venture,5800000.0,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,media,Entertainment
3,/organization/90min,venture,18000000.0,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,media,Entertainment
4,/organization/a-dance-for-me,equity_crowdfunding,1090000.0,A Dance for Me,Media|News|Photo Sharing|Video,operating,USA,media,Entertainment


In [42]:
# Examining the resultant data frame for number of observations

master_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88465 entries, 0 to 88464
Data columns (total 9 columns):
company_permalink     88465 non-null object
funding_round_type    88465 non-null object
raised_amount_usd     88465 non-null float64
name                  88465 non-null object
category_list         88465 non-null object
status                88465 non-null object
country_code          88465 non-null object
primary_sector        88465 non-null object
main_sector           88465 non-null object
dtypes: float64(1), object(8)
memory usage: 6.7+ MB


# Checkpoint 5: Sector Analysis 2

In [43]:
'''
writer = ExcelWriter('/Users/sdittaka/Downloads/master_frame.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
master_frame.to_excel(writer,'Sheet1')
writer.save()
'''

In [44]:
# Creating D1 frame for the observations of USA, funding type venture and the raised amount within the 5-15 million USD range

D1 = master_frame.loc[(master_frame['country_code'] == 'USA') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]

# The total number (or count) of investments for each main sector in a separate column

D1['main_sector_count'] = D1['main_sector'].apply(lambda x: D1.main_sector.value_counts()[x])

# The total amount invested in each main sector in a separate column

D1['main_sector_investment'] = D1['main_sector'].apply(lambda x: D1.groupby('main_sector')['raised_amount_usd'].sum()[x])

# Examining the data frame

D1.head()

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,main_sector_count,main_sector_investment
7,/organization/all-def-digital,venture,5000000.0,All Def Digital,Media,operating,USA,media,Entertainment,591,5.099198e+09
31,/organization/chefs-feed,venture,5000000.0,ChefsFeed,Media|Mobile|Restaurants|Technology,operating,USA,media,Entertainment,591,5.099198e+09
61,/organization/huffingtonpost,venture,5000000.0,The Huffington Post,Media|News|Publishing,acquired,USA,media,Entertainment,591,5.099198e+09
62,/organization/huffingtonpost,venture,5000000.0,The Huffington Post,Media|News|Publishing,acquired,USA,media,Entertainment,591,5.099198e+09
85,/organization/matchmine,venture,10000000.0,MatchMine,Media|News|Reviews and Recommendations,closed,USA,media,Entertainment,591,5.099198e+09


In [45]:
# Creating D2 frame for the observations of GBR, funding type venture and the raised amount within the 5-15 million USD range

D2 = master_frame.loc[(master_frame['country_code'] == 'GBR') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]

# The total number (or count) of investments for each main sector in a separate column

D2['main_sector_count'] = D2['main_sector'].apply(lambda x: D2.main_sector.value_counts()[x])

# The total amount invested in each main sector in a separate column

D2['main_sector_investment'] = D2['main_sector'].apply(lambda x: D2.groupby('main_sector')['raised_amount_usd'].sum()[x])

# Examining the data frame

D2.head()

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,main_sector_count,main_sector_investment
1,/organization/90min,venture,15000000.0,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,media,Entertainment,56,482784687.0
2,/organization/90min,venture,5800000.0,90min,Media|News|Publishing|Soccer|Sports,operating,GBR,media,Entertainment,56,482784687.0
225,/organization/common-interest-communities,venture,10000000.0,Common Interest Communities,Application Platforms|Internet|Software|Startups,operating,GBR,application platforms,"News, Search and Messaging",73,615746235.0
257,/organization/geospock-ltd-,venture,5460000.0,GeoSpock Ltd.,Application Platforms|Databases|Real Time,operating,GBR,application platforms,"News, Search and Messaging",73,615746235.0
258,/organization/geospock-ltd-,venture,5400000.0,GeoSpock Ltd.,Application Platforms|Databases|Real Time,operating,GBR,application platforms,"News, Search and Messaging",73,615746235.0


In [46]:
# Creating D2 frame for the observations of GBR, funding type venture and the raised amount within the 5-15 million USD range

D3 = master_frame.loc[(master_frame['country_code'] == 'IND') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]

# The total number (or count) of investments for each main sector in a separate column

D3['main_sector_count'] = D3['main_sector'].apply(lambda x: D3.main_sector.value_counts()[x])

# The total amount invested in each main sector in a separate column

D3['main_sector_investment'] = D3['main_sector'].apply(lambda x: D3.groupby('main_sector')['raised_amount_usd'].sum()[x])

# Examining the data frame

D3.head()

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,company_permalink,funding_round_type,raised_amount_usd,name,category_list,status,country_code,primary_sector,main_sector,main_sector_count,main_sector_investment
0,/organization/-fame,venture,10000000.0,#fame,Media,operating,IND,media,Entertainment,33,280830000.0
550,/organization/babajob,venture,10000000.0,Babajob,Curated Web|Information Technology|Services|St...,operating,IND,curated web,"News, Search and Messaging",52,433834545.0
593,/organization/bharat-matrimony,venture,11750000.0,Bharat Matrimony,Curated Web|Match-Making,operating,IND,curated web,"News, Search and Messaging",52,433834545.0
640,/organization/bluestone-com,venture,5000000.0,Bluestone.com,Curated Web,operating,IND,curated web,"News, Search and Messaging",52,433834545.0
642,/organization/bluestone-com,venture,10000000.0,Bluestone.com,Curated Web,operating,IND,curated web,"News, Search and Messaging",52,433834545.0


In [ ]:
'''
D1_Writer = ExcelWriter('/Users/sdittaka/Downloads/D1.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
D1.to_excel(D1_Writer,'Sheet1')
D1_Writer.save()

D2_Writer = ExcelWriter('/Users/sdittaka/Downloads/D2.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
D2.to_excel(D2_Writer,'Sheet1')
D2_Writer.save()

D3_Writer = ExcelWriter('/Users/sdittaka/Downloads/D3.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
D3.to_excel(D3_Writer,'Sheet1')
D3_Writer.save()
'''

### Total number of Investments (count)

In [47]:
D1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12063 entries, 7 to 88457
Data columns (total 11 columns):
company_permalink         12063 non-null object
funding_round_type        12063 non-null object
raised_amount_usd         12063 non-null float64
name                      12063 non-null object
category_list             12063 non-null object
status                    12063 non-null object
country_code              12063 non-null object
primary_sector            12063 non-null object
main_sector               12063 non-null object
main_sector_count         12063 non-null int64
main_sector_investment    12063 non-null float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.1+ MB


In [48]:
D2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 1 to 88265
Data columns (total 11 columns):
company_permalink         621 non-null object
funding_round_type        621 non-null object
raised_amount_usd         621 non-null float64
name                      621 non-null object
category_list             621 non-null object
status                    621 non-null object
country_code              621 non-null object
primary_sector            621 non-null object
main_sector               621 non-null object
main_sector_count         621 non-null int64
main_sector_investment    621 non-null float64
dtypes: float64(2), int64(1), object(8)
memory usage: 58.2+ KB


In [49]:
D3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 88157
Data columns (total 11 columns):
company_permalink         328 non-null object
funding_round_type        328 non-null object
raised_amount_usd         328 non-null float64
name                      328 non-null object
category_list             328 non-null object
status                    328 non-null object
country_code              328 non-null object
primary_sector            328 non-null object
main_sector               328 non-null object
main_sector_count         328 non-null int64
main_sector_investment    328 non-null float64
dtypes: float64(2), int64(1), object(8)
memory usage: 30.8+ KB


### Total amount of investment (USD)

In [50]:
print('Total amount of investment (USD) for USA is {} Million USD'.format(str(D1.raised_amount_usd.sum()/1000000)))

Total amount of investment (USD) for USA is 107757.097294 Million USD


In [51]:
print('Total amount of investment (USD) for GBR is {} Million USD'.format(str(D2.raised_amount_usd.sum()/1000000)))

Total amount of investment (USD) for GBR is 5379.078691 Million USD


In [52]:
print('Total amount of investment (USD) for IND is {} Million USD'.format(str(D3.raised_amount_usd.sum()/1000000)))

Total amount of investment (USD) for IND is 2949.543602 Million USD


### Top/Second/Third Sector name & no. of investments

In [53]:
# For USA, top 3 sectors (based on count of investments) and the corresponding count

D1.loc[:,['main_sector','main_sector_count']].sort_values(by='main_sector_count', ascending=False).drop_duplicates(keep='first', inplace=False)[0:3]

# Can also be obtained using => D1['main_sector'].value_counts()

,main_sector,main_sector_count
88457,Others,2950
52435,"Social, Finance, Analytics, Advertising",2714
75724,Cleantech / Semiconductors,2350


In [54]:
# For GBR, top 3 sectors (based on count of investments) and the corresponding count

D2.loc[:,['main_sector','main_sector_count']].sort_values(by='main_sector_count', ascending=False).drop_duplicates(keep='first', inplace=False)[0:3]

# Can also be obtained using => D2['main_sector'].value_counts()

,main_sector,main_sector_count
38526,Others,147
44688,"Social, Finance, Analytics, Advertising",133
50725,Cleantech / Semiconductors,130


In [55]:
# For IND, top 3 sectors (based on count of investments) and the corresponding count

D3.loc[:,['main_sector','main_sector_count']].sort_values(by='main_sector_count', ascending=False).drop_duplicates(keep='first', inplace=False)[0:3]

# Can also be obtained using => D3['main_sector'].value_counts()

,main_sector,main_sector_count
67060,Others,110
48445,"Social, Finance, Analytics, Advertising",60
2291,"News, Search and Messaging",52


### For point 3 (top sector count-wise), which company received the highest investment?

In [56]:
# For USA, "Virtustream" is the company which received the highest investment in "Others" sector

D1_others_df = D1.loc[D1.main_sector == 'Others']

D1_others_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
Virtustream,64300000.0
Capella Photonics,54968051.0
AirTight Networks,54201907.0
deCarta,52100000.0
Black Duck Software,51000000.0
Approva,50100000.0
Five9,49600000.0
Bit9,48433533.0
Aryaka Networks,48166500.0


In [57]:
# For GBR, "Electric Cloud" is the company which received the highest investment in "Others" sector

D2_others_df = D2.loc[D2.main_sector == 'Others']

D2_others_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
Electric Cloud,37000000.0
SenSage,36250000.0
Enigmatec,32500000.0
SilverRail Technologies,29000000.0
OpenCloud,27972766.0
MyOptique Group,27694613.0
Notonthehighstreet,26900000.0
ACS Clothing,26239014.0
Tribold,26000000.0


In [58]:
# For IND, "FirstCry.com" is the company which received the highest investment in "Others" sector

D3_others_df = D3.loc[D3.main_sector == 'Others']

D3_others_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
FirstCry.com,39000000.0
Myntra,38000000.0
CommonFloor,32900000.0
Pepperfry.com,28000000.0
ItzCash Card Ltd.,25000000.0
NxtGen Data Center & Cloud Services,22300000.0
Maharana Infrastructure and Professional Services Private Limited (MIPS),21600000.0
Comat Technologies,21000000.0
CaratLane,21000000.0


### For point 4 (second best sector count-wise), which company received the highest investment?

In [59]:
# For USA, "SST Inc. (Formerly ShotSpotter)" is the company which received the highest investment in "Social, Finance, Analytics, Advertising" sector

D1_SFAA_df = D1.loc[D1.main_sector == 'Social, Finance, Analytics, Advertising']

D1_SFAA_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
SST Inc. (Formerly ShotSpotter),67933006.0
Demandbase,63000000.0
Intacct,61800000.0
NetBase Solutions,60600000.0
Lotame,59700000.0
FirstRain,58344731.0
ChoiceStream,58300000.0
OpTier,55600000.0
Damballa,55000000.0


In [60]:
# For GBR, "Celltick Technologies" is the company which received the highest investment in "Social, Finance, Analytics, Advertising" sector

D2_SFAA_df = D2.loc[D2.main_sector == 'Social, Finance, Analytics, Advertising']

D2_SFAA_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
Celltick Technologies,37500000.0
myThings,34000000.0
Zopa,32900000.0
VisualDNA,28550000.0
MarketInvoice,25553007.0
SumUp,24293649.0
Amplience,24200000.0
Garlik,22350000.0
SportPursuit,22024487.0


In [61]:
# For IND, "Manthan Systems" is the company which received the highest investment in "Social, Finance, Analytics, Advertising" sector

D3_SFAA_df = D3.loc[D3.main_sector == 'Social, Finance, Analytics, Advertising']

D3_SFAA_df.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending = False)

,raised_amount_usd
name,
Manthan Systems,50700000.0
Komli Media,28000000.0
ShopClues.com,25000000.0
inTarvo,21900000.0
Grameen Financial Services,21556050.0
BankBazaar.com,19000000.0
Microland,18300000.0
Eka Software Solutions,16000000.0
Qyuki,15400000.0
